In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2

from collections import deque
from moviepy.editor import *
from IPython.core.display import HTML

%matplotlib inline

In [2]:
def preprocessFunc(imagesGen,actionsGen,slideSize=4):
    # image should be 150*600*3 nparray
    # action should be String 
    shortTermMemory = deque([],slideSize)
    vertices = np.array([[(430,30),(430,0),(600,0),(600,30)]]) 
    while True:
        img = next(imagesGen)
        act = next(actionsGen)
        
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # gray scale   (150,600)
        img = cv2.fillPoly(img,vertices,(255))     # override score
        img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.Canny(img,100,200)
#         img = cv2.resize(img,(120,30),interpolation=cv2.INTER_AREA) # (30,120)
        img = img[:,:,None] # (30,120) -> (30,120,1)
        
        shortTermMemory.append(img)
        
        if len(shortTermMemory) == slideSize:
            state  = np.concatenate(shortTermMemory,axis=2) # (30,120,4)
            action = act if act != 'Space' else 'Up'
            yield state,action

In [28]:
with open("human/human_0043.p",'rb') as f:
    imgs,acts = pickle.load(f)

In [29]:
imgGen = (img for img in imgs)
actGen = (act for act in acts)
preprocess = preprocessFunc(imgGen,actGen,3)
imgSeq = [img for img,act in preprocess]

In [30]:
clip = ImageSequenceClip(imgSeq, fps=50)
clip.write_videofile('preview.mp4')

[MoviePy] >>>> Building video preview.mp4
[MoviePy] Writing video preview.mp4


100%|█████████████████████████████████████████████████████████████████████████████| 1711/1711 [00:04<00:00, 369.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: preview.mp4 



In [31]:
HTML("""
    <video controls src="{0}" autoplay width=600px/> 
    """.format('preview.mp4')
)

In [32]:
print(acts)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Space', 'Space', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Space', 'Space', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Space', 'Space', 'Space', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Space', 'Space', 'Space', None, None, None, None, None, None, None, None, None, None, None, 'Space', 'Space', 'Space', None, None, None, None, None, None, None, None, None, None, 'Space', 'Space', 'Space', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [7]:
# fig,axes = plt.subplots(10,2,figsize=(32,64))

# for i in range(10):
#     index = np.random.randint(0,372)
#     axes[i,0].imshow(imgs[index])
#     axes[i,0].axis('off')
#     axes[i,0].text(400,30,acts[index],fontsize=12)
#     axes[i,1].imshow(imgs[index],cmap='gray')
#     axes[i,1].axis('off')